<img src="data/images/lecture-notebook-header.png" />

# Recommender Systems: Content-Based Systems

Content-based recommender systems are a type of recommendation system that suggest items to users based on the similarity of their content. These systems analyze the properties and features of items and recommend items that are similar to those that the user has expressed interest in or interacted with in the past.

The primary idea behind content-based recommendation is to build a profile or representation of both the items and the user's preferences. This is typically done by extracting relevant features or attributes from the items and creating a user profile based on their historical interactions. Here are the key steps involved in content-based recommendation:

* Item Representation: Each item in the system is described by a set of features or attributes. For example, in a movie recommendation system, the features could include genre, actors, director, and plot keywords.

* User Profile Creation: The system builds a profile for each user based on their interactions with items. This could include items they have rated, liked, purchased, or viewed. The user profile is created by aggregating the features of the items the user has interacted with.

* Similarity Calculation: A similarity measure is used to determine the similarity between the features of the items and the user profile. Various similarity metrics can be used, such as cosine similarity or Jaccard similarity.

* Recommendation Generation: The system identifies items that are most similar to the user's profile and recommends those items to the user. The items with the highest similarity scores are typically considered the most relevant and are presented as recommendations.

Content-based recommender systems have several advantages. They can provide personalized recommendations even for users with unique tastes or preferences, as they rely on item features rather than comparing users to other users. They can also offer explanations for the recommendations by highlighting the shared features between the recommended items and the user's profile.

However, content-based approaches have limitations as well. They may struggle to capture the complexity of user preferences, as they rely solely on item features and do not consider information about other users or social interactions. They can also suffer from the "filter bubble" effect, where users may receive recommendations that align closely with their existing interests, limiting exposure to new and diverse content.

## Setting up the Notebook

### Make all Required Imports

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from sklearn.metrics.pairwise import cosine_similarity

---

## Load & Prepare MovieLens Dataset (Small)

Throughout this notebook, we are using a small sample of [MovieLens](https://grouplens.org/datasets/movielens/) dataset. GroupLens Research has collected and made available rating data sets from the [MovieLens web site](https://movielens.org). Apart from the ratings, the dataset also comes with tags assigned to movies. The information about the movies include their id, title, and genre(s). Here are some key features of the MovieLens dataset:

* **Ratings:** The dataset includes ratings given by users to movies on a numerical scale, typically ranging from 1 to 5. Users provide their subjective ratings based on their personal preferences.

* **Movie Metadata:** MovieLens dataset also provides additional information about movies, such as genre, release year, and tags. This metadata can be used to build content-based recommendation models.

* **User Information:** The dataset contains demographic information about users, such as age, gender, occupation, and zip code. This information can be utilized to analyze user preferences and personalize recommendations.

* **Dataset Size:** The MovieLens dataset is available in different sizes. The smallest version, MovieLens 100K, contains 100,000 ratings from approximately 1,000 users on around 1,700 movies. Larger versions, such as MovieLens 1M, MovieLens 10M, and MovieLens 20M, contain respectively 1 million, 10 million, and 20 million ratings.

Content-based recommender systems assume that the items (here: movies) come with a set of features describing each item. In this notebook, to keep it simple, we limit ourselves to the genre of movies. We will see later why only using genres does not result in great recommendations. Including the tags would certainly improve this, but would also increase the complexity quite a bit.

### Load Movies

We first load all movies into a `pandas` DataFrame for further processing.

In [ ]:
# Read file using pandas
df_movies = pd.read_csv('data/datasets//ml-latest-small/movies.csv', sep=',', engine='python')

df_movies.head()

As we can see that all the genres of a movie are represented as a single string with the genres separated by a pipe symbol. So simplify subsequent steps, let's convert the dataframe into a dictionary with the movie ids as keys and the information about the movies as values. Each value is a 2-tuple with the movie title and the genres (but as set and all genres being lowercase).

In [ ]:
# Convert to dictionary {artist_id -> (movie_name, genres)}
movie_dict = { row[0]: (row[1], set(row[2].lower().split('|'))) for row in df_movies.values}

# Example for movie with id=1
print(movie_dict[1])

Later on, when we deal with the User-Item matrix, users and movies are identified by the row and column index in that matrix. For example the movie with id 1000 might be represented by the 700th column in the rating matrix To map back to the movies and users, we have to create mappings that allows us to map between movie and users ids and their respective row/column indices.

In [ ]:
movie_ids = df_movies['movieId'].unique()

movie_id2idx, movie_idx2id = {}, {}

for idx, movie_id in enumerate(movie_ids):
    movie_id2idx[movie_id] = idx
    movie_idx2id[idx] = movie_id

num_movies = len(movie_ids)

### Load Ratings

Now we can look at the ratings. Again, we first load the information into a `pandas` DataFrame.

In [ ]:
df_ratings = pd.read_csv('data/datasets/ml-latest-small/ratings.csv', sep=',', engine='python')

df_ratings.head()

Again, we have to create the mapping between the ids of users and there respective indices in the User-Item matrix.

In [ ]:
user_ids = df_ratings['userId'].unique()

user_id2idx, user_idx2id = {}, {}
 
for idx, user_id in enumerate(user_ids):
    user_id2idx[user_id] = idx
    user_idx2id[idx] = user_id

num_users = user_ids.shape[0]

After preprocessing both movies and ratings, we can get a sense of the size of our dataset.

In [ ]:
print('Number of users: {}'.format(num_users))
print('Number of movies: {}'.format(num_movies))

Of course, 610 users and 9,742 can hardly be considered a big dataset. But to get the basic ideas and concepts of content-based recommender systems, it's more than sufficient.

### Initialize User-Item Matrix R

The core information of recommender systems is the user-item matrix R. For our use case, the items are the movies and the matrix elements represent the rating of a user about a movie.

In [ ]:
R = np.zeros((num_users, num_movies))

print('The User-Item Matrix R has shape of {}'.format(R.shape))

Obviously, all elements in R are currently 0. Now we can go through ratings in `df_ratings` to fill. Note how we need to map the user and movie ids to valid matrix indices.

In [ ]:
for index, row in tqdm(df_ratings.iterrows(), total=len(df_ratings)):
    user_id, movie_id, rating = row['userId'], row['movieId'], row['rating']

    # Convert movie and user ids to indices 
    user_idx = user_id2idx[user_id]
    movie_idx = movie_id2idx[movie_id]
    
    # Fill matrix at the right spot with the rating
    R[user_idx][movie_idx] = rating

Let's check how sparse matrix R is, i.e., what percentage of entries are non-zero.

In [ ]:
num_nonzero = np.count_nonzero(R > 0)
percent_sparsity = num_nonzero / np.prod(R.shape) * 100

print('Number of non-zero entries in M: {} (sparsity: {:.3f}%)'.format(num_nonzero, percent_sparsity))

A sparsity level of around 1.7% is actually rather high. In real-world settings with many more users and movies, the sparsity is typically much lower than that. This obviously calls for more efficient data structures to store very sparse matrices, but that's beyond the scope of this notebook.

### Building the Movie Profiles

The core idea of content-based recommender systems is to represent the items using a set of features describing the items to allow for, e.g., the calculation of the similarities between the items. As mentioned above, we use the genres of a movie to describe each movie and will later discuss why this is not a great choice.

#### Identifying all Movie Genres

We first need to know how many genres are there overall to know the number of features for each movie. To do this, we simply create a union of all the genres of all movies.

In [ ]:
all_genres = set()

for movie_id, movie_data in movie_dict.items():
    # Update current set by adding the genres of the current movie
    all_genres.update(movie_data[1])
    
# Convert set to sorted list
all_genres = sorted(all_genres)    
    
    
print('Number of movie genres: {}\n'.format(len(all_genres)))
print(all_genres)

#### Encoding

The method `one_hot_encode_genres()` simply takes the genres of a given movie and generates a one-hot encoding -- in more detail, a vector of size 20 (since there a 20 genres) with 1s at the position if the movie belongs to the respective genre. The code cell includes an example.

In [ ]:
def one_hot_encode_genres(all_genres, movie_genres):
    v = np.zeros((len(all_genres, )))
    for idx, genre in enumerate(all_genres):
        if genre in movie_genres:
            v[idx] = 1
    return v

one_hot_encode_genres(all_genres, {'comedy', 'sci-fi'})

The following code cell creates a `numpy` matrix containg the one-hot vectors for all movies. The shape of this matrix is (9742, 20) with 9,742 being the number of movies and 20 being the number of features.

In [ ]:
movie_profiles = np.zeros((num_movies, len(all_genres)))

for movie_id, movie_data in tqdm(movie_dict.items(), total=len(movie_dict)):
    # Encode genres of movie to a one-hot vector
    genre_vector = one_hot_encode_genres(all_genres, movie_data[1])
    # Update profile matrix
    movie_profiles[movie_id2idx[movie_id]] = genre_vector

We now have the important information in the right shape to start generating recommendations.

## Approach 1: Pairwise Item Similarities

The most basic and intuitive approach is to compute all pairwise similarities between the movies. Then, given a movie -- for example the movie user watched last, rated highest, watched most frequently, etc. -- we can find the most similar movies and present those recommendations

With [`cosine_similarity()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) provides a handy method to calculate all pairwise similarities for us:

In [ ]:
movie_similarities = cosine_similarity(movie_profiles, movie_profiles)

print(movie_similarities.shape)

Naturally, the shape of this matrix is `(num_movies, num_movies)`. The following examples of matrix elements are purely for a very basic sanity check: the similarity between the same movie should be 1, and the similarities should be symmetric.

In [ ]:
print(movie_similarities[0,0]) # Should be 1 (apart from precision issues)
print(movie_similarities[0,1]) # These two should return the same value
print(movie_similarities[1,0]) # due to the symmetry of the distance matrix

Using this matrix with all pairwise similarities between the movies, we can now define a method that recommends `topk` movies given a "reference" movie specified by its id `movie_id`. The method mainly returns the most similar movies but makes some extensions to make the recommendations a bit more realistic:

* If specified, the input movie will be removed from the recommendations (in practice, this is not always done; for example, YouTube often recommends videos you have already seen)

* There is a small notion of randomization by picking the `topk` movies randomly from the list of `topk*factor` most similar movies. Otherwise the recommendations would always be the same.

In [ ]:
def get_recommendations_pis(movie_id, topk=10, factor=2, remove_input_movie=True):
    
    # Given the movie id, get the correct index in the matrix
    movie_idx = movie_id2idx[movie_id]
    
    # Get the topk*factor mos similar movies (identified by their indices)
    most_similar_movie_indices = np.argsort(movie_similarities[movie_idx])[::-1][:topk*factor]
    
    ## Remove the input movie from the list of most similar movies (optional)
    if remove_input_movie == True:
        # Get the index of input movie id (might return an empty array)
        idx = np.where(most_similar_movie_indices == movie_idx)[0]
        # Remove movie id from list of most similar movies, if present.
        most_similar_movie_indices = np.delete(most_similar_movie_indices, idx)
    
    ## Pick a random topk sample of topk*factor most similar movies
    most_similar_movie_indices = np.random.choice(most_similar_movie_indices, size=topk, replace=False)    
    
    ## Get the ids of the movies from the indicies of the movies
    recommended_movie_ids = [ movie_idx2id[idx] for idx in  most_similar_movie_indices ]
    
    ## Return the recommendations as ordered list of movie ids
    return recommended_movie_ids 
    

You can try different movies to see what kind of recommendations are returned. Due to the randomization even for the same movie, the recommendations should slightly differ between different runs. Of course, you can enforce the same recommendation for the same movie by setting `factor=1`.

In [ ]:
recommended_movie_ids = get_recommendations_pis(112852) # Guardians of the Galaxy
#recommended_movie_ids = get_recommendations_pis(4896) # Harry Potter 1
#recommended_movie_ids = get_recommendations_pis(480) # Jurassic Park

for rank, movie_id in enumerate(recommended_movie_ids):
    print('[Rank {}] {}'.format(rank+1, movie_dict[movie_id]))

While the recommendations seem intuitive, they also show the limitations of this approach when using only the genres as features to describe the movie. It assumes that a user might like, for example, an action/sci-fi movie if he or she had liked or seen another action/sci-fi. However, movies of the same genre(s) can often vary significantly regarding their quality and popularity. In practice, the set of features should contain more indicative attributes such as the director, actors, box offices numbers, etc.

-------------------------------------

## Approach 2: User-Item Similarities

The second approach we talked about in the lecture utilized the similarities between users and items. This approach requires us to represent users similar to the movies. This means for our use case that we have to represent a user as a vector of size 20.

Let's first initialize each user profile as zero-vectors.

In [ ]:
user_profiles = np.zeros((num_users, movie_profiles.shape[1]))

print(user_profiles.shape)

We saw in the lecture that we need to normalize the ratings w.r.t, to users average user ratings. Firstly, all ratings are positive (1-5). This means that there's no explicit notion of "dislike" and unrated movies (0) would be treated as rated worst. And secondly, different users might have different notions of what rating represents a good movie. For example, one user might rate a good movie with 3 or higher, while another user considers good movies only from 4 upwards.

The following code cell calculated the average rating for each user. Note that we have to exclude unrated movies (rating=0) which would otherwise distort the average ratings. We can do this by "masking" the 0-ratings so they are not considered when computing the averages.

In [ ]:
## Mask all unknown ratings
masked = np.ma.masked_equal(R, 0)

## Compute the average user ratings
user_mean_rankings = masked.mean(axis=1).reshape(-1, 1)

The following code computes the profiles for all users using the approach presented in the lectures. The computations includes some extension to improve the recommendations in a meaningful but yet simple manner:

* Consider only genres a user has watched a sufficient number of times. For example, if a user rated only a single horror movie this rating should not affect the user's profile.

The code below contains more details that should make the individual steps clear.

In [ ]:
## Minimum number of ratings a movie with a certain genres needs to have received
min_rating_count = 10

for user_idx in tqdm(range(num_users)):

    ## Get all the movies that the user has rated
    rated_movies_indices = np.where(R[user_idx] != 0)[0]
    
    ## Get all movie profiles for the movies the user has rated
    rated_movie_profiles = movie_profiles[rated_movies_indices]

    ## Find all genres which the user has not rated enough
    ## This gives "cleaner" results as it avoid outliers
    invalid_feature_indices = np.where(np.count_nonzero(rated_movie_profiles, axis=0) < min_rating_count)

    ## Set the genre information if "rare genres" to 0
    ## (so they have not effect on the user profile)
    rated_movie_profiles[:,invalid_feature_indices] = 0
    
    ## Get the ratings of all the movies the user has rated
    ratings = R[user_idx][rated_movies_indices]    
    
    ## Normalize the ratings
    ratings_normalized = ratings - user_mean_rankings[user_idx]
    
    ## We have to multiple rating for each genre -- this makes is convenient
    ratings_repeated = np.repeat(ratings_normalized, len(all_genres)).reshape(-1, len(all_genres))
    r  = ratings_repeated * rated_movie_profiles

    ## Calculate the counts of how often the user has rated a certain genre
    counts = np.sum(rated_movie_profiles, axis=0)
    
    ## Calucate the final user profiles
    user_profile = np.sum(r, axis=0) / counts
    
    # As previous division can cause dividing by 0, we have to fix this
    user_profile = np.ma.fix_invalid(user_profile, copy=True, fill_value=0).data
    
    # Set user profile into user profiles matrix
    user_profiles[user_idx] = user_profile

Now we have a profile for each user in the same format as the movies, i.e., a vector of size 20. This allows us not to again calculate the distances between all users and movies.

In [ ]:
user_movie_distances = cosine_similarity(user_profiles, movie_profiles)

print(user_movie_distances.shape)

Based on this new similarity matrix, we can define another method to return recommendations, but this time having a `user_id` as input instead of a `movie_id`. Now the best recommendations are the movies with profiles most similar to the profile of the user. Again we refine this basic idea a bit:

* If specified, all movies the user has already rated will be removed from the recommendations.
* We add some randomization by choosing the `topk` recommendations randomly from the set of `topk*factor` most similar movies.

In [ ]:
def get_recommendations_uis(user_id, topk=10, factor=2, remove_rated=True):
    ## Get the user index given the user id
    user_idx = user_id2idx[user_id]
    
    ## Find the topk*factor movies closest to the profile of the given user
    recommendations = np.argsort(user_movie_distances[user_idx].squeeze())
    
    ## Remove movies from recommendations the user has already rated
    if remove_rated == True:
        # Find all indicies of movie the user has already rated
        already_rated_movies = np.where(R[user_idx] != 0)[0]
        # Remove those indicies from the current recommendations
        recommendations = np.delete(recommendations, already_rated_movies)
        
    recommendations = recommendations[::-1][:topk*factor]

    ## Pick a random topk sample of topk*factor recommendations
    recommendations = np.random.choice(recommendations, size=topk, replace=False)
    
    ## Return the indices (sorted) for all recommended movies
    return np.array([ movie_idx2id[r] for r in recommendations ])
    

Let's look at the recommendation for the user with `user_id=1`; feel free to change this to other users.

In [ ]:
recommended_movie_ids = get_recommendations_uis(1, factor=2)

for rank, movie_id in enumerate(recommended_movie_ids):
    print('[Rank {}] {}'.format(rank+1, movie_dict[movie_id]))

Looking at the recommendations for the user with `user_id=1` we can tell that the user seems to like kids movies and animation movies. Note that -- compare the recommendation based on pairwise movie similarities -- here the recommended movies do not cover the exact same genres.

---

## Summary

Content-based recommender systems are a type of recommendation system that leverage the properties and features of items to make personalized recommendations to users. These systems analyze the content of items, such as movies, products, or articles, and recommend items that are similar to those the user has shown interest in or interacted with. One of the main advantages of content-based recommender systems is their ability to provide personalized recommendations based on the specific features and attributes of items. This enables them to cater to users with unique tastes and preferences, as the recommendations are not reliant on comparisons with other users. Additionally, content-based approaches can provide explanations for their recommendations by highlighting the shared features between the recommended items and the user's profile. This transparency can enhance user trust and understanding.

However, content-based recommender systems also have limitations. They may struggle to capture the complexity of user preferences since they rely solely on item features and do not consider information about other users or social interactions. This can lead to a lack of serendipity in recommendations, where users are not exposed to new and diverse content. Furthermore, content-based approaches may face challenges in accurately extracting and representing the relevant features of items, especially in domains with nuanced or subjective characteristics. Lastly, the scalability of content-based systems can be a concern, as creating item profiles and computing similarity measures for large datasets can be computationally expensive.

To address these limitations, hybrid recommender systems that combine content-based and collaborative filtering techniques are commonly employed. By leveraging the strengths of both approaches, hybrid systems can provide more accurate and diverse recommendations by incorporating user-item interactions alongside item features. This combination allows for a balance between personalization and exploration, offering users a wider range of options while still considering their individual preferences.

In conclusion, content-based recommender systems excel at providing personalized recommendations based on item features, offering transparency through explanations, and catering to users with unique preferences. However, they may struggle to capture complex user preferences, limit exposure to diverse content, face challenges in feature extraction, and encounter scalability issues. Hybrid recommender systems that blend content-based and collaborative filtering approaches provide a promising solution to overcome these limitations and offer more accurate and diverse recommendations to users.